In [ ]:
import sys
sys.path.append('../')

In [1]:
#from utils.intermediate_layer_extraction import return_layer_input
import deepmatcher as dm

In [20]:
hybrid_model = dm.MatchingModel(attr_summarizer='hybrid')
hybrid_model.load_state('../models/itunesamazon_hybrid.pth')

In [21]:
from deepmatcher.data import MatchingIterator
from torch.autograd import Variable

In [22]:
def _return_input(module,module_input,module_output):
    global current_layer_input
    current_layer_input = Variable(module_input[0].data.cuda(),requires_grad=True)


    
def _flat_list(l):
    flat_list = []
    for sublist in l:
        for item in sublist:
            flat_list.append(item)
    return flat_list


def return_layer_input(dataset_dir,dataset_name,batch_size,model,layer,device = 0):
    dataset = dm.data.process(path=dataset_dir,train=dataset_name+'.csv',left_prefix='ltable_',right_prefix='rtable_',cache=dataset_name+'.pth')
    dataset_tuple = dataset,
    splits = MatchingIterator.splits(dataset_tuple,batch_size=batch_size, device = device)
    tupleids = []
    layer_inputs = []
    hook = layer.register_forward_hook(_return_input)
    for batch in splits[0]:
        tupleids.append(batch.id)
        model.forward(batch)
        layer_inputs.append(current_layer_input)
    hook.remove()
    return layer_inputs,list(map(int,_flat_list(tupleids)))


In [23]:
neg_classifier_hybrid,neg_ids = return_layer_input('../Structured/itunes-amazon','negatives',128,hybrid_model,
                                                                        hybrid_model.classifier,device=-1)

In [25]:
hook = hybrid_model.attr_summarizers.Song_Name.register_forward_hook(print)
hook.remove()

In [35]:
dataset = dm.data.process(path='../Structured/itunes-amazon',train='negatives.csv',
                          left_prefix='ltable_',right_prefix='rtable_',cache='negatives.pth')
dataset_tuple = dataset,
splits = MatchingIterator.splits(dataset_tuple,batch_size=128, device = -1)
tupleids = []
layer_inputs = []
hook = hybrid_model.attr_summarizers.Song_Name.register_forward_hook(print)
batches = []
for batch in splits[0]:
    batches.append(batch)
hybrid_model.forward(batches[0])
hook.remove()

Hybrid(
  (word_contextualizer): RNN(
    (rnn_groups): ModuleList(
      (0): GRU(300, 150, batch_first=True, bidirectional=True)
    )
    (dropouts): ModuleList(
      (0): Dropout(p=0)
    )
    (bypass_networks): ModuleList(
      (0): None
    )
    (input_dropout): NoMeta(
      (module): Dropout(p=0)
    )
  )
  (word_comparator): Attention(
    (alignment_networks): ModuleList(
      (0): AlignmentNetwork(
        (transform): Transform(
          (transforms): ModuleList(
            (0): Linear(in_features=300, out_features=300, bias=True)
            (1): Linear(in_features=300, out_features=300, bias=True)
          )
          (bypass_networks): ModuleList(
            (0): Bypass(
              (highway_gate): Linear(in_features=300, out_features=300, bias=True)
            )
            (1): Bypass(
              (highway_gate): Linear(in_features=300, out_features=300, bias=True)
            )
          )
        )
      )
    )
    (value_merge): Merge(
    )
    (com

RuntimeError: Variable data has to be a tensor, but got Variable